In [1]:
from functools import partial

import numpy as np
import statsmodels.api as sm

from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly, sklearn_sm)

from sklearn.model_selection import (train_test_split, cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone

## 5.3.1 Validation set approach

In [2]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

In [3]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [4]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred) ** 2)

23.61661706966988

In [5]:
def evalMSE(terms, response, train, test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]

    results = sm.OLS(y_train, X_train).fit()
    test_preds = results.predict(X_test)

    return np.mean((y_test - test_preds)**2)

In [6]:
MSE = np.zeros(3)

for i, degree in enumerate(range(1, 4)):
    MSE[i] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)

MSE

array([23.61661707, 18.76303135, 18.79694163])

In [7]:
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=3)

MSE = np.zeros(3)

for i, degree in enumerate(range(1, 4)):
    MSE[i] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)

MSE

array([20.75540796, 16.94510676, 16.97437833])

## 5.3.2 Cross-validation

### LOOCV

In [8]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))

X, y, = Auto.drop(columns=['mpg']), Auto['mpg']
# LOOCV k=n
cv_results = cross_validate(hp_model, X, y, cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792924

In [9]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)

for i, d in enumerate(range(1, 6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, y, cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])

cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443033, 19.03323827])

In [10]:
np.arange(2)

array([0, 1])

In [11]:
np.power.outer(H[:2], np.arange(2))

array([[  1, 130],
       [  1, 165]])

In [12]:
np.power.outer(H[:2], np.arange(3))

array([[    1,   130, 16900],
       [    1,   165, 27225]])

In [13]:
130**2

16900

In [14]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

### K-fold CV

In [15]:
cv_error = np.zeros(5)

# use same splits for each degree
cv = KFold(n_splits=10, shuffle=True, random_state=0)

for i, d in enumerate(range(1, 6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, y, cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])

cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720581])

### ShuffleSplit

In [16]:
# n_splits=1
validation = ShuffleSplit(n_splits=1, test_size=196, random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv=validation)
results['test_score']

array([23.61661707])

In [17]:
# n_splits=10
validation = ShuffleSplit(n_splits=10, test_size=196, random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv=validation)

results['test_score'].mean(), results['test_score'].std()

(23.802232661034168, 1.421845094109185)

## 5.3.3 Bootstrap

### Estimating the accuracy of a statistic of interest

In [18]:
Portfolio = load_data('Portfolio')
Portfolio.head()

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983


Global Minimum Variance (GMV)
$$
\hat{\alpha} \;=\; 
\frac{\hat{\sigma}_Y^2 \;-\; \hat{\sigma}_{XY}}
     {\hat{\sigma}_X^2 \;+\; \hat{\sigma}_Y^2 \;-\; 2\,\hat{\sigma}_{XY}}
$$

In [19]:
def alpha_func(df, i):
    cov_ = np.cov(df[['X', 'Y']].loc[i], rowvar=False)
    return ((cov_[1, 1] - cov_[0, 1]) /
            (cov_[0, 0] + cov_[1, 1] - 2*cov_[0, 1]))

In [20]:
alpha_func(Portfolio, range(100))

0.57583207459283

In [21]:
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace=True))

0.6074452469619004

Bootstrap standard error

In [22]:
def boot_SE(func, df, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or df.shape[0]

    for _ in range(B):
        i = rng.choice(df.index, n, replace=True)
        value = func(df, i)
        first_ += value
        second_ += value**2

    return np.sqrt(second_ / B - (first_ / B)**2)

In [23]:
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
alpha_SE

0.09118176521277699

### Estimating the accuracy of a linear regression model

In [24]:
def boot_OLS(model_mat, resp, df, idx):
    df_ = df.loc[idx];
    y_ = df_[resp]
    X_ = clone(model_mat).fit_transform(df_)
    return sm.OLS(y_, X_).fit().params

In [25]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

In [26]:
rng.choice(Auto.index, 2, replace=True)

array(['ford ltd', 'toyota corolla tercel'], dtype=object)

In [27]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto, rng.choice(Auto.index, 392, replace=True)) 
          for _ in range(10)])

array([[39.12226577, -0.1555926 ],
       [37.18648613, -0.13915813],
       [37.46989244, -0.14112749],
       [38.56723252, -0.14830116],
       [38.95495707, -0.15315141],
       [39.12563927, -0.15261044],
       [38.45763251, -0.14767251],
       [38.43372587, -0.15019447],
       [37.87581142, -0.1409544 ],
       [37.95949036, -0.1451333 ]])

In [28]:
hp_se = boot_SE(hp_func, Auto, B=1000, seed=10)
hp_se

intercept     0.731176
horsepower    0.006092
dtype: float64

In [29]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [30]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

In [31]:
quad_model = MS([poly('horsepower', 2, raw=True)])
M = sm.OLS(Auto['mpg'], quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64